In [14]:
import numpy as np
import pandas as pd
import sklearn as sk
# from sklearn.trees import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [15]:

df = pd.read_csv("traffic.csv",parse_dates=True, index_col='DateTime')


# extract year from date
df['Year'] = pd.Series(df.index).apply(lambda x: x.year).to_list()

# extract month from date
df['Month'] = pd.Series(df.index).apply(lambda x: x.month).to_list()

# extract day from date
df['Day'] = pd.Series(df.index).apply(lambda x: x.day).to_list()

# extract hour from date
df['Hour'] = pd.Series(df.index).apply(lambda x: x.hour).to_list()

# grouped = df.groupby(df.Junction)
# df1 = grouped.get_group(1)
# df2 = grouped.get_group(2)
# df3 = grouped.get_group(3)
# df4 = grouped.get_group(4)





In [16]:
# print(df)



ratio = 80
dataDivide = len(df) * ratio // 100

X = df[["Junction", 'Year',  'Month' , 'Day'  ,'Hour']].values.tolist()
y = sk.utils.validation.column_or_1d(df.iloc[:, 1:2].values.tolist())
# print(x1)
# print(x2)
# from sklearn import preprocessing
# import numpy as np
# x2 = preprocessing.normalize([x2])[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=43)
# x1Train, x1Test, x2Train, x2Test = x1[:dataDivide], x1[
#     dataDivide:], x2[:dataDivide], x2[dataDivide:]

In [36]:
y_train

array([  6,   6,  25, ...,  31,  11, 122])

In [18]:
# from sklearn import svm
# regr = svm.SVR()
# regr.fit(X_train, y_train)

In [32]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

clf = make_pipeline(StandardScaler(), SVR())
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svr', SVR())])

In [20]:
# model = SVC(random_state=1)
# model.fit(x1Train, x2Train)
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
clf1 = make_pipeline(StandardScaler(), RandomForestRegressor())
clf1.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor())])

In [37]:
# print(rf.score(X_test, y_test))0.9517263752046465
print(clf1.score(X_test, y_test))
print(clf.score(X_test, y_test))
print(clf1.predict([[4, 2017, 4, 17, 16]]))
print(clf.predict([[4, 2017, 4, 17, 16]]))

0.9517263752046465
0.817708490026077
[6.43]
[8.61660924]
